In [1]:
import requests
from time import sleep

# 利用API获取用户ID

In [4]:
def get_user_info(uid):
    result=requests.get('https://m.weibo.cn/api/container/getIndex?type=uid&value={}'.format(uid))
    json_data=result.json()
    print(json_data)
get_user_info('3655689037')#1350995007

{'ok': 1, 'data': {'userInfo': {'id': 3655689037, 'screen_name': '欢迎新用户', 'profile_image_url': 'https://tva1.sinaimg.cn/crop.0.0.179.179.180/d9e5634djw1east9pi6bej2050050dfw.jpg', 'profile_url': 'https://m.weibo.cn/u/3655689037?uid=3655689037&luicode=10000011&lfid=1005053655689037', 'statuses_count': 5077, 'verified': False, 'verified_type': -1, 'close_blue_v': False, 'description': '', 'gender': 'f', 'mbtype': 2, 'urank': 12, 'mbrank': 1, 'follow_me': False, 'following': False, 'followers_count': 1608992, 'follow_count': 0, 'cover_image_phone': 'https://tva1.sinaimg.cn/crop.0.0.640.640.640/549d0121tw1egm1kjly3jj20hs0hsq4f.jpg', 'avatar_hd': 'https://ww1.sinaimg.cn/orj480/d9e5634djw1east9pi6bej2050050dfw.jpg', 'like': False, 'like_me': False, 'toolbar_menus': [{'type': 'profile_follow', 'name': '关注', 'pic': '', 'params': {'uid': 3655689037}}, {'type': 'link', 'name': '聊天', 'pic': 'http://h5.sinaimg.cn/upload/2015/06/12/2/toolbar_icon_discuss_default.png', 'params': {'scheme': 'sinaweib

In [14]:
def get_user_info(uid):
    result=requests.get('https://m.weibo.cn/api/container/getIndex?type=uid&value={}'.format(uid))
    json_data=result.json()
    
    if json_data['ok']==1:
        userInfo={
            'user_id':json_data['data']['userInfo']['id'],
            'screen_name':json_data['data']['userInfo']['screen_name'],
            'profile_image_url':json_data['data']['userInfo']['profile_image_url'],
            #'gender':(json_data['data']['userInfo']['gender']=='f'?'女':'男'),
            'followers_count':json_data['data']['userInfo']['followers_count'],
            'follow_count':json_data['data']['userInfo']['follow_count'],
            'verified':json_data['data']['userInfo']['verified'],
            #'verified_reason':(verified?json_data['data']['userInfo']['verified_reason']:'')
            'containerid': json_data['data']['tabsInfo']['tabs'][1]['containerid']   # 此字段在获取博文中需要
        }
        if json_data['data']['userInfo']['gender']=='f':
            gender='女'
        else:
            gender='男'
        
        if userInfo['verified']:
            verified_reason=json_data['data']['userInfo']['verified_reason']
        else:
            verified_reason=''
        userInfo['gender']=gender
        userInfo['verified_reason']=verified_reason
#    print(userInfo)
        
        
        
#get_user_info('1350995007')

{'user_id': 1350995007, 'screen_name': '我是娜扎', 'profile_image_url': 'https://tvax2.sinaimg.cn/crop.20.0.1202.1202.180/50868c3fly8fhzdg7xwnfj20yi0xejtv.jpg', 'followers_count': 15690226, 'follow_count': 516, 'verified': True, 'containerid': '1076031350995007', 'gender': '女', 'verified_reason': '演员，代表作《择天记》'}


In [41]:
def get_all_posts(uid,containerid):
    page=0
    posts=[]
    while page<20:
        result=requests.get('https://m.weibo.cn/api/container/getIndex?type=uid&value={}&containerid={}&page={}'
                              .format(str(uid), str(containerid), str(page)))
        json_data=result.json()
        # 当博文获取完毕，退出循环
        if not json_data['data']['cards']:
            break
        #print(json_data['data']['cards'][0]['mblog']['text'])
        for x in json_data['data']['cards']:
            if x['card_type']==9:               #不是所有的元素都有mblog键,只有card_type==9的有, 不加上判断条件会报错
                posts.append(x['mblog']['text'])
        #for x in json_data['data']['cards']:
        #    posts.append(x['mblog']['text'])
        page+=1
        sleep(0.5)
    return posts
posts=get_all_posts(1350995007,1076031350995007)

# html2text
html2text API:
https://pypi.org/project/html2text/
直接使用html2text.html2text()函数就行了
jieba.analyse API:
https://github.com/fxsjy/jieba/blob/master/README.md
### 基于 TF-IDF 算法的关键词抽取
import jieba.analyse

jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())
sentence 为待提取的文本
topK 为返回几个 TF/IDF 权重最大的关键词，默认值为 20
withWeight 为是否一并返回关键词权重值，默认值为 False
allowPOS 仅包括指定词性的词，默认值为空，即不筛选
jieba.analyse.TFIDF(idf_path=None) 新建 TFIDF 实例，idf_path 为 IDF 频率文件
代码示例 （关键词提取）

https://github.com/fxsjy/jieba/blob/master/test/extract_tags.py

关键词提取所使用逆向文件频率（IDF）文本语料库可以切换成自定义语料库的路径

用法： jieba.analyse.set_idf_path(file_name) # file_name为自定义语料库的路径
自定义语料库示例：https://github.com/fxsjy/jieba/blob/master/extra_dict/idf.txt.big
用法示例：https://github.com/fxsjy/jieba/blob/master/test/extract_tags_idfpath.py
关键词提取所使用停止词（Stop Words）文本语料库可以切换成自定义语料库的路径

用法： jieba.analyse.set_stop_words(file_name) # file_name为自定义语料库的路径
自定义语料库示例：https://github.com/fxsjy/jieba/blob/master/extra_dict/stop_words.txt
用法示例：https://github.com/fxsjy/jieba/blob/master/test/extract_tags_stop_words.py
关键词一并返回关键词权重值示例

用法示例：https://github.com/fxsjy/jieba/blob/master/test/extract_tags_with_weight.py
### 基于 TextRank 算法的关键词抽取
jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v')) 直接使用，接口相同，注意默认过滤词性。
jieba.analyse.TextRank() 新建自定义 TextRank 实例
算法论文： TextRank: Bringing Order into Texts

基本思想:
将待抽取关键词的文本进行分词
以固定窗口大小(默认为5，通过span属性调整)，词之间的共现关系，构建图
计算图中节点的PageRank，注意是无向带权图
使用示例:
见 test/demo.py

词性标注
jieba.posseg.POSTokenizer(tokenizer=None) 新建自定义分词器，tokenizer 参数可指定内部使用的 jieba.Tokenizer 分词器。jieba.posseg.dt 为默认词性标注分词器。
标注句子分词后每个词的词性，采用和 ictclas 兼容的标记法。
用法示例
>>> import jieba.posseg as pseg
>>> words = pseg.cut("我爱北京天安门")
>>> for word, flag in words:
...    print('%s %s' % (word, flag))
...
我 r
爱 v
北京 ns
天安门 ns
并行分词
原理：将目标文本按行分隔后，把各行文本分配到多个 Python 进程并行分词，然后归并结果，从而获得分词速度的可观提升

基于 python 自带的 multiprocessing 模块，目前暂不支持 Windows

用法：

jieba.enable_parallel(4) # 开启并行分词模式，参数为并行进程数
jieba.disable_parallel() # 关闭并行分词模式
例子：https://github.com/fxsjy/jieba/blob/master/test/parallel/test_file.py

实验结果：在 4 核 3.4GHz Linux 机器上，对金庸全集进行精确分词，获得了 1MB/s 的速度，是单进程版的 3.3 倍。

注意：并行分词仅支持默认分词器 jieba.dt 和 jieba.posseg.dt。

Tokenize：返回词语在原文的起止位置
注意，输入参数只接受 unicode
默认模式
result = jieba.tokenize(u'永和服装饰品有限公司')
for tk in result:
    print("word %s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))
word 永和                start: 0                end:2
word 服装                start: 2                end:4
word 饰品                start: 4                end:6
word 有限公司            start: 6                end:10

搜索模式
result = jieba.tokenize(u'永和服装饰品有限公司', mode='search')
for tk in result:
    print("word %s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))
word 永和                start: 0                end:2
word 服装                start: 2                end:4
word 饰品                start: 4                end:6
word 有限                start: 6                end:8
word 公司                start: 8                end:10
word 有限公司            start: 6                end:10

In [48]:
import jieba.analyse
from html2text import html2text
content=' '.join([html2text(x) for x in posts])
result=jieba.analyse.textrank(content, topK=1000, withWeight=True)
keyword={}
for x in result:
    keyword[x[0]]=x[1]
#print(keyword)

# 构建词云
wordcloud API:
https://github.com/amueller/word_cloud
https://github.com/amueller/word_cloud/blob/master/examples/simple.py
wordcloud没有中文只能够自己加载字体此处使用simhei.ttf

In [55]:
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import numpy as np
alice_mask=np.array(Image.open('alice_color.png'))
#f=open('stopwords.txt','r')
#stopword=
#stopwords暂时没有实现
wc=WordCloud(font_path='simhei.ttf',background_color="white", max_words=2000, mask=alice_mask, max_font_size=100, random_state=42)
wc.generate_from_frequencies(keyword)
image_colors = ImageColorGenerator(alice_mask)

In [1]:
import matplotlib.pyplot as plt
# 显示图片
plt.imshow(wc)
plt.imshow(wc.recolor(color_func=image_colors))
plt.axis("off") # 关闭图像坐标系
plt.show()

NameError: name 'wc' is not defined